In [2]:
import pandas as pd
import pytz
import pendulum

cities = ["Chicago", "New_York", "London", "Berlin", "Moscow", "Dubai", "Shanghai", "Tokyo"]
cities_west_of_berlin = 3
start_time_chicago = 17

timezone_cities = [ [tz for tz in pytz.all_timezones if city in tz][0] for city in cities]
pendulum_for_each_city = [pendulum.today(tz) for tz in timezone_cities]
p_chicago = pendulum_for_each_city[0]
offset_cities = [-p_chicago.diff(pendulum_for_each_city[i]).in_hours() if i<cities_west_of_berlin else p_chicago.diff(pendulum_for_each_city[i]).in_hours() for i in range(len(pendulum_for_each_city))]

df = pd.DataFrame({_: [(start_time_chicago+o+_)%24 for o in offset_cities] for _ in range(24)})
df.rename(index={_: cities[_] for _ in range(len(cities))}, inplace=True)
pd.options.display.max_columns = None
display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Chicago,17,18,19,20,21,22,23,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
New_York,16,17,18,19,20,21,22,23,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
London,12,13,14,15,16,17,18,19,20,21,22,23,0,1,2,3,4,5,6,7,8,9,10,11
Berlin,23,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
Moscow,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0
Dubai,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0,1
Shanghai,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0,1,2,3,4,5
Tokyo,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0,1,2,3,4,5,6
